In [8]:
import pandas as pd
import re

In [10]:
#reads the data
with open('PyScripts/TextFiles/Raw_data_2023.txt', 'r', encoding='utf-8') as file:
    content = file.read()

In [11]:
df = pd.DataFrame([content],columns=['Content'])

df

,Content
0,"\nSoderblog\nSEEN, READ 2023\nMr/Dr Soderbergh..."


In [12]:
split_content= df.iloc[0,0].split('\n')
new_df = pd.DataFrame(split_content, columns=['Content'])


In [ ]:
new_df = new_df.iloc[10:].reset_index(drop=True)

In [13]:
new_df["Dates"] = new_df["Content"].str.extract(r'(\d{2}/\d{2})')
new_df["Content"] = new_df["Content"].apply(lambda x: x[5:])
new_df["Content"] = new_df["Content"].str.replace(r' data-mce-fragment="1"', "")
new_df["Content"] = new_df["Content"].str.replace(r'(</em>,)[^,]*', r'\1', regex=True)

new_df

,Content,Dates
0,,NaN
1,blog,NaN
2,READ 2023,NaN
3,Soderbergh @ 2024-01-03,NaN
4,"aps, bold: <strong>MOVIE</strong>",NaN
...,...,...
273,"<em>Purple Hibiscus</em>,, <strong>NEITHER CO...",12/28
274,"<em>Candace Pert</em>,, HOUSE HUNTERS INTERNA...",12/29
275,"<em>Genealogy of a Murder</em>,",12/30
276,"<strong>ORIGIN</strong>, DAVE CHAPPELLE: THE ...",12/31


In [14]:
new_df = new_df.assign(Content = new_df.Content.str.split('</em>')).explode("Content")
new_df = new_df.assign(Content = new_df.Content.str.split('</strong>')).explode("Content")
new_df = new_df.assign(Content = new_df.Content.str.split(',')).explode("Content")

new_df

,Content,Dates
0,,NaN
1,blog,NaN
2,READ 2023,NaN
3,Soderbergh @ 2024-01-03,NaN
4,aps,NaN
...,...,...
276,,12/31
276,DAVE CHAPPELLE: THE DREAMER,12/31
276,<strong>MICHAEL CLAYTON,12/31
276,,12/31


In [15]:
new_df = new_df[new_df["Content"] != ""]
new_df

,Content,Dates
1,blog,NaN
2,READ 2023,NaN
3,Soderbergh @ 2024-01-03,NaN
4,aps,NaN
4,bold: <strong>MOVIE,NaN
...,...,...
274,<strong>GRAN TURISMO,12/29
275,<em>Genealogy of a Murder,12/30
276,<strong>ORIGIN,12/31
276,DAVE CHAPPELLE: THE DREAMER,12/31


In [16]:
new_df["Bold"] = new_df["Content"].apply(lambda x: 1 if '<strong>' in x else 0)
new_df["Italic"] = new_df["Content"].apply(lambda x: 1 if '<em>' in x else 0)
new_df["Short"] = new_df["Content"].apply(lambda x: 1 if '*' in x else 0)
new_df

C:\Users\sarbend\AppData\Local\Temp\ipykernel_8108\2733650213.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df["Bold"] = new_df["Content"].apply(lambda x: 1 if '<strong>' in x else 0)
C:\Users\sarbend\AppData\Local\Temp\ipykernel_8108\2733650213.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df["Italic"] = new_df["Content"].apply(lambda x: 1 if '<em>' in x else 0)
C:\Users\sarbend\AppData\Local\Temp\ipykernel_8108\2733650213.py:3: SettingWithCopyWarning: 
A value is trying to be set on a 

,Content,Dates,Bold,Italic,Short
1,blog,NaN,0,0,0
2,READ 2023,NaN,0,0,0
3,Soderbergh @ 2024-01-03,NaN,0,0,0
4,aps,NaN,0,0,0
4,bold: <strong>MOVIE,NaN,1,0,0
...,...,...,...,...,...
274,<strong>GRAN TURISMO,12/29,1,0,0
275,<em>Genealogy of a Murder,12/30,0,1,0
276,<strong>ORIGIN,12/31,1,0,0
276,DAVE CHAPPELLE: THE DREAMER,12/31,0,0,0


In [17]:
new_df["Content"] = new_df["Content"].str.replace("<em>", "")
new_df["Content"] = new_df["Content"].str.replace("<strong>", "")
new_df["AllCaps"] = new_df["Content"].apply(lambda x: 1 if x.isupper() else 0)
new_df

C:\Users\sarbend\AppData\Local\Temp\ipykernel_8108\2828787279.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df["Content"] = new_df["Content"].str.replace("<em>", "")
C:\Users\sarbend\AppData\Local\Temp\ipykernel_8108\2828787279.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df["Content"] = new_df["Content"].str.replace("<strong>", "")
C:\Users\sarbend\AppData\Local\Temp\ipykernel_8108\2828787279.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFra

,Content,Dates,Bold,Italic,Short,AllCaps
1,blog,NaN,0,0,0,0
2,READ 2023,NaN,0,0,0,1
3,Soderbergh @ 2024-01-03,NaN,0,0,0,0
4,aps,NaN,0,0,0,0
4,bold: MOVIE,NaN,1,0,0,0
...,...,...,...,...,...,...
274,GRAN TURISMO,12/29,1,0,0,1
275,Genealogy of a Murder,12/30,0,1,0,0
276,ORIGIN,12/31,1,0,0,1
276,DAVE CHAPPELLE: THE DREAMER,12/31,0,0,0,1


In [12]:
def has_quoted_string(text):
        # Check if there's any content between quotation marks
        pattern = r'\u201c([^\u201d]*)\u201d'
        return 1 if re.search(pattern, str(text)) else 0

new_df["Quoted"] = new_df["Content"].apply(lambda x: has_quoted_string(x))
new_df

,Content,Dates,Bold,Italic,Short,AllCaps,Quoted
1,BELOW DECK,01/01,0,0,1,1,0
1,PRINCE ANDREW THE MUSICAL,01/01,0,0,1,1,0
1,THE BEST OF COMEDY 2022,01/01,0,0,1,1,0
2,Tomorrow,01/03,0,1,1,0,0
2,and Tomorrow,01/03,0,0,1,0,0
...,...,...,...,...,...,...,...
264,GRAN TURISMO,12/29,1,0,1,1,0
265,Genealogy of a Murder,12/30,0,1,1,0,0
266,ORIGIN,12/31,1,0,1,1,0
266,DAVE CHAPPELLE: THE DREAMER,12/31,0,0,1,1,0


In [18]:
new_df.to_csv('Content2024.csv', index=False)